## Import Dependencies

In [13]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from config import username, password


## Write SQL Queries
### 1: Create Database

In [8]:
create_db_query = 'CREATE DATABASE MovieDB'

### 2: Create Table Schema

In [9]:
create_schema_query = 'CREATE SCHEMA IF NOT EXISTS ETL'

### 3. Create Tables

In [58]:
ETL_actors = '''DROP TABLE IF EXISTS actors_role;
                CREATE TABLE actors_role(id INT PRIMARY KEY
                                   ,names VARCHAR
                                   ,last_role VARCHAR)'''
ETL_movies_info = '''CREATE TABLE movies_info(movie_id INT PRIMARY KEY
                                             ,title VARCHAR
                                             ,plot  VARCHAR
                                             ,IMDb_rating NUMERIC
                                             ,main_cast_id INT
                                             ,main_cast_name VARCHAR
                                             ,other_cast_id INT
                                             ,other_cast_name VARCHAR
                                             )'''

### 4: Create "View" that is our "Data Warehouse" Table

In [ ]:
#create_DW_view =

## Establish the Connection

In [11]:
conn = psycopg2.connect(database="postgres", user=username, password=password, host='localhost', port= '5432')
conn.autocommit = True    #or we could commit with conn.commit()
    #f"user={username} password={password}");
#con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

## Create a Cursor Object Because Reasons

In [6]:
cursor = conn.cursor()

## Create Database

In [12]:
#Creating a database
cursor.execute(create_db_query)
print("Database created successfully........")

Database created successfully........


## Create Schema and Tables

In [59]:
cursor.execute(ETL_actors)
print("actor table created successfully........")

actor table created successfully........


In [55]:
cursor.execute(ETL_movies_info)
print("movies_info table created successfully........")

movies_info table created successfully........


## Test Data Insert

In [41]:
import pandas as pd
import csv

In [28]:
actor_df = pd.read_csv("actor_dummy.csv")
actor_df

,id,names,last role
0,352835,Brahim Hadjadj,Buamama
1,552483,Jean Martin,Dossier: disparus
2,754272,Yacef Saadi,The Battle of Algiers
3,449197,Samia Kerbash,The Battle of Algiers
4,657597,Ugo Paletti,The Battle of Algiers
5,252678,Fusia El Kader,The Battle of Algiers
6,69934,Mohamed Ben Kassen,The Battle of Algiers
7,605398,Franco Moruzzi,I guerrieri dell'anno 2072
8,626221,Tommaso Neri,The Battle of Algiers
9,746250,Rouïched,Adhilai al beida


In [38]:
actor_df.iloc[0,]

#for x in range(10):
#    print(actor_df.iloc[x,1])
for row, value in actor_df.iterrows():
    print(row, value)

0 id                   352835
names        Brahim Hadjadj
last role           Buamama
Name: 0, dtype: object
1 id                      552483
names              Jean Martin
last role    Dossier: disparus
Name: 1, dtype: object
2 id                          754272
names                  Yacef Saadi
last role    The Battle of Algiers
Name: 2, dtype: object
3 id                          449197
names                Samia Kerbash
last role    The Battle of Algiers
Name: 3, dtype: object
4 id                          657597
names                  Ugo Paletti
last role    The Battle of Algiers
Name: 4, dtype: object
5 id                          252678
names               Fusia El Kader
last role    The Battle of Algiers
Name: 5, dtype: object
6 id                           69934
names           Mohamed Ben Kassen
last role    The Battle of Algiers
Name: 6, dtype: object
7 id                               605398
names                    Franco Moruzzi
last role    I guerrieri dell'anno 2072
N

In [ ]:
insert_part = "INSERT INTO actors VALUES (%s, %s, %s)"
cursor.execute(insert_part, (row1_list))
#insert query = "INSERT INTO actors VALUES {}".format("(000000, 'Aranza Dumbledore', 'The Lion King')")
cursor.copy_from

## Use the "with open" loop to go straight from CSV to SQL

In [45]:
with open('actor_dummy.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cursor.execute(
        "INSERT INTO actors VALUES (%s, %s, %s)",
        row
        )

In [47]:
act = cursor.execute("select * from actors")

In [49]:
result = cursor.fetchall()
print(result)

[(352835, 'Brahim Hadjadj', 'Buamama'), (552483, 'Jean Martin', 'Dossier: disparus'), (754272, 'Yacef Saadi', 'The Battle of Algiers'), (449197, 'Samia Kerbash', 'The Battle of Algiers'), (657597, 'Ugo Paletti', 'The Battle of Algiers'), (252678, 'Fusia El Kader', 'The Battle of Algiers'), (69934, 'Mohamed Ben Kassen', 'The Battle of Algiers'), (605398, 'Franco Moruzzi', "I guerrieri dell'anno 2072"), (626221, 'Tommaso Neri', 'The Battle of Algiers'), (746250, 'Rouïched', 'Adhilai al beida'), (921881, 'Gene Wesson', 'The Battle of Algiers')]


In [ ]:
csv_name = 'actor_dummy.csv'
table = "actors_role"
with open(csv_name, 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        cursor.execute(
        "INSERT INTO {table} VALUES (%s, %s, %s)",
        row
        )

In [63]:
csv_path = 'actor_dummy.csv'
table = "actors_role"

def load_table(csv_path, table):
    with open(csv_path, 'r') as f:
        reader = csv.reader(f)
        next(reader) # Skip the header row.
        fstring = f"INSERT INTO {table} VALUES (%s, %s, %s)"
        for row in reader:
            cursor.execute(
            fstring,
            row
            )

In [64]:
load_table(csv_path, table)

In [66]:
cursor.execute("SELECT * FROM actors_role")
result = cursor.fetchall()
print(result)

[(352835, 'Brahim Hadjadj', 'Buamama'), (552483, 'Jean Martin', 'Dossier: disparus'), (754272, 'Yacef Saadi', 'The Battle of Algiers'), (449197, 'Samia Kerbash', 'The Battle of Algiers'), (657597, 'Ugo Paletti', 'The Battle of Algiers'), (252678, 'Fusia El Kader', 'The Battle of Algiers'), (69934, 'Mohamed Ben Kassen', 'The Battle of Algiers'), (605398, 'Franco Moruzzi', "I guerrieri dell'anno 2072"), (626221, 'Tommaso Neri', 'The Battle of Algiers'), (746250, 'Rouïched', 'Adhilai al beida'), (921881, 'Gene Wesson', 'The Battle of Algiers')]


In [ ]:
#Closing the connection
conn.close()